In [1]:
import numpy as np
import matplotlib.pyplot as plt

from toygrad import MLP, Layer, Sigmoid, SquaredError

np.random.seed(2137)

CLF = "data/classification"
REG = "data/regression"

def one_hot(y):
    y = y.astype(int)
    encoded = np.zeros((y.size, y.max()+1))
    encoded[np.arange(y.size),y] = 1
    return encoded

def read_data(path):
    data = np.genfromtxt(path, delimiter=',')[1:]
    X = data[:, :-1]
    Y = one_hot(data[:, -1])
    return X, Y

In [2]:
layers = [
    Layer(in_size=2, out_size=5, activ_function=Sigmoid),
    Layer(in_size=5, out_size=5, activ_function=Sigmoid),
    Layer(in_size=5, out_size=3, activ_function=Sigmoid),
]
m = MLP(
    layers=layers,
    loss=SquaredError,
    bias=True,
    batch_size=16,
    epochs=5,
    momentum=0.001,
    learning_rate=0.1,
)

In [3]:
X, Y = read_data(f"{CLF}/data.simple.test.100.csv")

In [4]:
m.train(X, Y)

=========== Epoch 0 ============
Batch 1/6    16/100 - loss: 0.530
Batch 2/6    32/100 - loss: 0.522
Batch 3/6    48/100 - loss: 0.514
Batch 4/6    64/100 - loss: 0.509
Batch 5/6    80/100 - loss: 0.498
Batch 6/6   100/100 - loss: 0.487
=========== Epoch 1 ============
Batch 1/6    16/100 - loss: 0.475
Batch 2/6    32/100 - loss: 0.462
Batch 3/6    48/100 - loss: 0.449
Batch 4/6    64/100 - loss: 0.442
Batch 5/6    80/100 - loss: 0.421
Batch 6/6   100/100 - loss: 0.402
=========== Epoch 2 ============
Batch 1/6    16/100 - loss: 0.382
Batch 2/6    32/100 - loss: 0.365
Batch 3/6    48/100 - loss: 0.346
Batch 4/6    64/100 - loss: 0.342
Batch 5/6    80/100 - loss: 0.317
Batch 6/6   100/100 - loss: 0.297
=========== Epoch 3 ============
Batch 1/6    16/100 - loss: 0.279
Batch 2/6    32/100 - loss: 0.270
Batch 3/6    48/100 - loss: 0.256
Batch 4/6    64/100 - loss: 0.268
Batch 5/6    80/100 - loss: 0.246
Batch 6/6   100/100 - loss: 0.233
=========== Epoch 4 ============
Batch 1/6    16/100